In [1]:
!pip install librosa soundfile numpy pandas scikit-learn tensorflow matplotlib

In [5]:
!unzip archive.zip -d content

Archive:  archive.zip
  inflating: content/Actor_01/03-01-01-01-01-01-01.wav  
  inflating: content/Actor_01/03-01-01-01-01-02-01.wav  
  inflating: content/Actor_01/03-01-01-01-02-01-01.wav  
  inflating: content/Actor_01/03-01-01-01-02-02-01.wav  
  inflating: content/Actor_01/03-01-02-01-01-01-01.wav  
  inflating: content/Actor_01/03-01-02-01-01-02-01.wav  
  inflating: content/Actor_01/03-01-02-01-02-01-01.wav  
  inflating: content/Actor_01/03-01-02-01-02-02-01.wav  
  inflating: content/Actor_01/03-01-02-02-01-01-01.wav  
  inflating: content/Actor_01/03-01-02-02-01-02-01.wav  
  inflating: content/Actor_01/03-01-02-02-02-01-01.wav  
  inflating: content/Actor_01/03-01-02-02-02-02-01.wav  
  inflating: content/Actor_01/03-01-03-01-01-01-01.wav  
  inflating: content/Actor_01/03-01-03-01-01-02-01.wav  
  inflating: content/Actor_01/03-01-03-01-02-01-01.wav  
  inflating: content/Actor_01/03-01-03-01-02-02-01.wav  
  inflating: content/Actor_01/03-01-03-02-01-01-01.wav  
  inflati

In [6]:
dataset_path= "/content/content/audio_speech_actors_01-24"

In [7]:
import os
for root, dirs, files in os.walk(dataset_path):
  for file in files:
    print(file)

03-01-04-01-01-01-09.wav
03-01-08-02-01-02-09.wav
03-01-04-01-01-02-09.wav
03-01-03-01-01-01-09.wav
03-01-06-02-02-01-09.wav
03-01-07-02-01-01-09.wav
03-01-05-02-02-01-09.wav
03-01-05-02-02-02-09.wav
03-01-08-01-01-02-09.wav
03-01-06-01-01-01-09.wav
03-01-05-01-01-01-09.wav
03-01-03-01-01-02-09.wav
03-01-07-02-01-02-09.wav
03-01-01-01-01-01-09.wav
03-01-08-01-01-01-09.wav
03-01-04-02-02-02-09.wav
03-01-01-01-02-01-09.wav
03-01-02-02-02-01-09.wav
03-01-06-01-02-01-09.wav
03-01-05-01-01-02-09.wav
03-01-04-02-02-01-09.wav
03-01-07-01-02-02-09.wav
03-01-02-01-01-01-09.wav
03-01-07-01-01-02-09.wav
03-01-06-02-01-02-09.wav
03-01-06-01-02-02-09.wav
03-01-02-02-01-01-09.wav
03-01-06-02-01-01-09.wav
03-01-07-02-02-02-09.wav
03-01-08-02-02-02-09.wav
03-01-04-02-01-02-09.wav
03-01-05-01-02-02-09.wav
03-01-08-01-02-01-09.wav
03-01-02-02-01-02-09.wav
03-01-01-01-02-02-09.wav
03-01-07-02-02-01-09.wav
03-01-02-01-02-01-09.wav
03-01-07-01-01-01-09.wav
03-01-06-01-01-02-09.wav
03-01-03-02-01-02-09.wav


# **Extracting Features from .wav files: **

In [70]:
import librosa
import numpy as np
emotion_map = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}
def extract_features(dataset_path):
  y, sr = librosa.load(dataset_path)
  mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
  mfcc = (mfcc -np.mean(mfcc))/np.std(mfcc)
  return mfcc.T
features =[]
labels=[]

Here we are trying to access the actor folders , thus access the files (voice recordings) the audio data is labeled ( file names contain numbers that refer to the diffferent type of emotions that exist in the audio)

In [71]:
for actor_folder in os.listdir(dataset_path):
  actor_path = os.path.join(dataset_path, actor_folder)
  for file in os.listdir(actor_path):
    if file.endswith(".wav"):
      file_path = os.path.join(actor_path, file)
      emotion_code= file.split("-")[2]
      label = emotion_map[emotion_code]
      if label:
        mfcc =extract_features(file_path)
        features.append(mfcc)
        labels.append(label)

# **Encode Labels + Split Data ( for training and testing)**

In [72]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [73]:
#Encoding Labels
encoder = LabelEncoder()
y = encoder.fit_transform(labels)
y = to_categorical(y)

# Find the maximum length of the features
max_length = max([mfcc.shape[0] for mfcc in features])

# Pad the features to the maximum length
padded_features = []
for mfcc in features:
    # Calculate padding required
    padding_length = max_length - mfcc.shape[0]
    # Pad the array with zeros
    padded_mfcc = np.pad(mfcc, ((0, padding_length), (0, 0)), mode='constant')
    padded_features.append(padded_mfcc)

# Convert the padded features to a NumPy array
x = np.array(padded_features)

In [74]:
#splitting data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# **Building and training the Model:**

In [75]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten

As we are using the CNN, we will be setting the number of hidden layers which are 3 for these layer  we will use ReLU as an activation function (of their neurons)  and for the last layer (output)  we will use the softmax activation function.

In [77]:
num_classes = 8
mfcc= (mfcc -np.mean(mfcc))/np.std(mfcc)
max_length= max(mfcc.shape[0] for mfcc in features)
time_steps= max_length
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(time_steps, 40 , 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#Now the training!
model.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 19s 431ms/step - accuracy: 0.1828 - loss: 2.1285 - val_accuracy: 0.2882 - val_loss: 1.8521
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 15s 419ms/step - accuracy: 0.3415 - loss: 1.7358 - val_accuracy: 0.3299 - val_loss: 1.6701
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 15s 429ms/step - accuracy: 0.4136 - loss: 1.5340 - val_accuracy: 0.3681 - val_loss: 1.6556
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 20s 407ms/step - accuracy: 0.4851 - loss: 1.4021 - val_accuracy: 0.4132 - val_loss: 1.5419
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 21s 423ms/step - accuracy: 0.5618 - loss: 1.2277 - val_accuracy: 0.4549 - val_loss: 1.4891
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 21s 445ms/step - accuracy: 0.6033 - loss: 1.0890 - val_accuracy: 0.4931 - val_loss: 1.3373
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 19s 397ms/step - accuracy: 0.6730 - loss: 0.8878 - val_accuracy: 0.4965 - val_loss: 1.3410
Epoch 8/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 14s 397ms/step - accuracy: 0.6878 - loss: 0.8416 - val_accu

# Model Evaluation:

In [78]:
loss , accuracy = model.evaluate(x_test, y_test)
print(f"Accuracy: {accuracy*100:.2f} %")

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - accuracy: 0.6367 - loss: 1.7666
Accuracy: 61.46 %


# **Predict Label(emotions) for a new Audio**

In [79]:
audio_Path= "/content/content/audio_speech_actors_01-24/Actor_01/03-01-01-01-01-01-01.wav"

In [80]:
def predict_emotion(audio_path):
    mfcc = extract_features(audio_path)
    mfcc = mfcc.reshape(1, -1)
    label=encoder.inverse_transform([np.argmax(model)])
    return label[0]
print(predict_emotion(audio_Path))

angry
